# load model


In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # select GPU

# cav-mae 的整体模型定义并导入预训练权重

In [4]:
import os
import torch
import timm
from src.models import CAVMAE

assert timm.__version__ == '0.4.5' # it is important to have right version of timm

model_path = 'pretrained_model/audio_model.pth'
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7" # select GPU
# CAV-MAE model with decoder
#print(torch.cuda.device_count())
torch.cuda.set_device(1)
audio_model = CAVMAE(audio_length=1024,
                     modality_specific_depth=11,
                     norm_pix_loss=True, tr_pos=False) # most models are trained with pixel normalization and non-trainabe positional embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl_weight = torch.load(model_path, map_location=device)
audio_model = torch.nn.DataParallel(audio_model) # it is important to convert the model to dataparallel object as all weights are saved in dataparallel format (i.e., in module.xxx)
miss, unexpected = audio_model.load_state_dict(mdl_weight, strict=False)
print('miss=',miss, 'unexpected=', unexpected) # check if all weights are correctly loaded

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
Number of Audio Patches: 512, Visual Patches: 196
Audio Positional Embedding Shape: torch.Size([1, 512, 768])
Visual Positional Embedding Shape: torch.Size([1, 196, 768])
miss= ['module.blocks_maedfer.patch_embed.proj.weight', 'module.blocks_maedfer.patch_embed.proj.bias', 'module.blocks_maedfer.blocks.0.norm1.weight', 'module.blocks_maedfer.blocks.0.norm1.bias', 'module.blocks_maedfer.blocks.0.attn.q_bias', 'module.blocks_maedfer.blocks.0.attn.v_bias', 'module.blocks_maedfer.blocks.0.attn.qkv.weight', 'module.blocks_maedfer.blocks.0.attn.proj.weight', 'module.blocks_maedfer.blocks.0.attn.proj.bias', 'module.blocks_maedfer.blocks.0.norm2.weight', 'module.blocks_maedfer.blocks.0.norm2.bias', 'module.blocks_maedfer.blocks.0.mlp.fc1.weight', 'module.blocks_maedfer.blocks.0.mlp.fc1.bias', 'module.blocks_maedfer.blocks.0.mlp.fc2.weight', 'module.blocks_maedfer.blocks.0.mlp.fc2.bias', 'module.blocks_maedfer.block

# 定义cav-mae模型，查看其中的video模块输出维度

In [24]:
import os
import torch
import timm
from src.models.my_model import CAVMAE

cav_mae_model = CAVMAE(audio_length=1024, 
                       modality_specific_depth=11,
                       norm_pix_loss=True, tr_pos=False)

A CAV-MAE Model
Use norm_pix_loss:  True
Learnable Positional Embedding:  False
video encoder PretrainVisionTransformerEncoder(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,)

In [25]:
Video_data = torch.randn(8, 3, 16, 224, 224)
Audio_data = torch.randn(8, 128, 1024)
output = cav_mae_model(Audio_data, Video_data)

Patch size = (16, 16)
image shape: torch.Size([8, 3, 16, 224, 224])
mask shape: (1568,)
batchsize 8
mask torch.Size([8, 1568])
after video encoder torch.Size([8, 128, 768])
a.shape torch.Size([8, 128, 768])
x.shape torch.Size([8, 256, 768])


NameError: name 'ids_restore_v' is not defined

# 定义mae dfer模型

In [8]:
from src.models import modeling_finetune
import torch
from src.models.modeling_finetune import vit_base_patch16_160

# 定义模型
model1 = vit_base_patch16_160()
model1 = model1.to('cuda:0')
ckt = torch.load('pretrained_model/audio_model.pth', map_location='cuda:0')
print(ckt.keys())

msg = model1.load_state_dict(ckt, strict=False)
print(msg)

odict_keys(['module.modality_a', 'module.modality_v', 'module.pos_embed_a', 'module.pos_embed_v', 'module.mask_token', 'module.decoder_modality_a', 'module.decoder_modality_v', 'module.decoder_pos_embed_a', 'module.decoder_pos_embed_v', 'module.patch_embed_a.proj.weight', 'module.patch_embed_a.proj.bias', 'module.patch_embed_v.proj.weight', 'module.patch_embed_v.proj.bias', 'module.blocks_a.0.norm1.weight', 'module.blocks_a.0.norm1.bias', 'module.blocks_a.0.norm1_a.weight', 'module.blocks_a.0.norm1_a.bias', 'module.blocks_a.0.norm1_v.weight', 'module.blocks_a.0.norm1_v.bias', 'module.blocks_a.0.attn.qkv.weight', 'module.blocks_a.0.attn.qkv.bias', 'module.blocks_a.0.attn.proj.weight', 'module.blocks_a.0.attn.proj.bias', 'module.blocks_a.0.norm2.weight', 'module.blocks_a.0.norm2.bias', 'module.blocks_a.0.norm2_a.weight', 'module.blocks_a.0.norm2_a.bias', 'module.blocks_a.0.norm2_v.weight', 'module.blocks_a.0.norm2_v.bias', 'module.blocks_a.0.mlp.fc1.weight', 'module.blocks_a.0.mlp.fc1.bi

# 用maedfer 进行一次前向

In [4]:

# 创建随机输入数据
input_data = torch.rand(8, 3, 16, 160, 160)

# 进行前向传播
output1, output1_feature = model1(input_data,save_feature=True)
model1.reset_classifier( num_classes=10)
print(output1_feature.shape)
print(output1.shape)


torch.Size([1, 3, 16, 160, 160])
torch.Size([1, 3, 16, 160, 160])
torch.Size([1, 768])
torch.Size([1, 1000])


In [14]:
from src.models.modeling_pretrain import PretrainVisionTransformer
model2 = PretrainVisionTransformer()
input_data = torch.rand(8, 3, 16, 224, 224)
input_mask = torch.rand(8, 16, 160, 160)
output2, output2_feature = model2(input_data, input_mask)

TypeError: ~ (operator.invert) is only implemented on integer and Boolean-type tensors